In [32]:
from neuroposelib import read
from neuroposelib import vis
import numpy as np
import time
from IPython.display import Video
from pathlib import Path
import matplotlib.pyplot as plt
# %matplotlib inline
from neuroposelib import preprocess
from neuroposelib import write
from neuroposelib import features
from neuroposelib import DataStruct as ds
from neuroposelib.embed import Embed
from neuroposelib import analysis
import pandas as pd
from neuroposelib.embed import Watershed

config = read.config('/home/lq53/mir_repos/dappy_24_nov/byws_version/mir_1.yaml')

connectivity = read.connectivity(
    path=config["skeleton_path"], skeleton_name=config["skeleton_name"]
)

# Make out_path
Path(config["out_path"]).mkdir(parents=True, exist_ok=True)

# meta, meta_by_frame = read.meta(config["data_path"] + "demo_meta.csv", id=ids)
pose, ids, meta, meta_by_frame = read.pose_from_meta(
    path=config["meta_path"], connectivity=connectivity, key="Prediction_path", file_type="dannce"
)

Path(config["out_path"]).mkdir(parents=True, exist_ok=True)

write.pose_h5(pose,ids, config['out_path'] + 'pose_merged_newcol.h5')


pose = preprocess.rotate_spine(preprocess.center_spine(pose, keypt_idx=4), keypt_idx=[4, 3])

rel_vel, rel_vel_labels = features.get_velocities(
    pose,
    ids,
    connectivity.joint_names,
    joints=np.delete(np.arange(18), 4),
    widths=[5, 11, 51],
    f_s = 30
)
write.features_h5(rel_vel, rel_vel_labels, path=config["out_path"] + "velocity.h5")

angles, angle_labels = features.get_angles(pose, connectivity.angles)

# Angular Velocities
angular_vel, angular_vel_labels = features.get_angular_vel(
        angles,
        angle_labels,
        ids,
        widths=[5, 11, 51],
    )
write.features_h5(
    angular_vel, angular_vel_labels, path=config["out_path"] + "angular_velocity.h5"
)

ego_pose, labels = features.get_ego_pose(pose, connectivity.joint_names)

# Write
write.features_h5(angles, angle_labels, path=config["out_path"] + "angels.h5")
write.features_h5(ego_pose, labels, path=config["out_path"] + "ego_pose.h5")

# Compute and save Euler angles (if applicable)
euler_angles, euler_labels = features.get_euler_angles(pose, connectivity.angles)
write.features_h5(
    euler_angles, euler_labels, path=config["out_path"] + "euler_angles.h5"
)

# Compute and save head angular velocities
head_angular = features.get_head_angular(pose, ids, widths=[5, 10, 50])
write.features_h5(
    head_angular, ["head_angular"], path=config["out_path"] + "head_angular.h5"
)


print("saved")
# del angles, angle_labels
# del ego_pose, labels
# del rel_vel, rel_vel_labels
# del head_angular, euler_angles, euler_labels

0it [00:00, ?it/s]

115it [00:06, 17.43it/s]


Centering poses to mid spine ...
Rotating spine to xz plane ... 
Detected centered pose input - calculating relative velocities ... 


100%|██████████| 115/115 [01:16<00:00,  1.50it/s]


KeyboardInterrupt: 

In [33]:
rel_vel, rel_vel_labels = features.get_velocities(
    pose,
    ids,
    connectivity.joint_names,
    joints=np.delete(np.arange(18), 4),
    widths=[5, 11, 51],
    f_s = 30
)
write.features_h5(rel_vel, rel_vel_labels, path=config["out_path"] + "velocity.h5")

angles, angle_labels = features.get_angles(pose, connectivity.angles)

# Angular Velocities
angular_vel, angular_vel_labels = features.get_angular_vel(
        angles,
        angle_labels,
        ids,
        widths=[5, 11, 51],
    )
write.features_h5(
    angular_vel, angular_vel_labels, path=config["out_path"] + "angular_velocity.h5"
)

ego_pose, labels = features.get_ego_pose(pose, connectivity.joint_names)

# Write
write.features_h5(angles, angle_labels, path=config["out_path"] + "angels.h5")
write.features_h5(ego_pose, labels, path=config["out_path"] + "ego_pose.h5")

# Compute and save Euler angles (if applicable)
euler_angles, euler_labels = features.get_euler_angles(pose, connectivity.angles)
write.features_h5(
    euler_angles, euler_labels, path=config["out_path"] + "euler_angles.h5"
)

# Compute and save head angular velocities
head_angular = features.get_head_angular(pose, ids, widths=[5, 10, 50])
write.features_h5(
    head_angular, ["head_angular"], path=config["out_path"] + "head_angular.h5"
)

Detected centered pose input - calculating relative velocities ... 


100%|██████████| 115/115 [01:15<00:00,  1.52it/s]


Calculating joint angles ... 


100%|██████████| 55/55 [00:04<00:00, 11.57it/s]


Calculating velocities of angles ... 


100%|██████████| 115/115 [00:51<00:00,  2.24it/s]


Reformatting pose to egocentric pose features ... 
Calculating joint angles - Euler ... 


  4%|▎         | 2/55 [00:01<00:35,  1.50it/s]/home/lq53/mir_repos/dappy_24_nov/neuroposelib/src/neuroposelib/features.py:165: RuntimeWarning: invalid value encountered in divide
  v2_u = v2[:, plane_dict[key]] / np.expand_dims(
  9%|▉         | 5/55 [00:02<00:14,  3.35it/s]/home/lq53/mir_repos/dappy_24_nov/neuroposelib/src/neuroposelib/features.py:162: RuntimeWarning: invalid value encountered in divide
  v1_u = v1[:, plane_dict[key]] / np.expand_dims(
100%|██████████| 115/115 [00:01<00:00, 68.25it/s]


In [36]:
# Ensure all feature datasets have the same number of samples
assert rel_vel.shape[0] == angular_vel.shape[0] == angles.shape[0] == ego_pose.shape[0]

# Combine features
combined_features = np.hstack([rel_vel, angular_vel, angles, ego_pose])

# Concatenate labels
combined_labels = rel_vel_labels + angular_vel_labels + angle_labels + labels



Calculating principal components ... 


NameError: name 'tqdm' is not defined

In [42]:
from tqdm import tqdm
from typing import Optional, Union, List, Tuple, Type
def pca(
    features: np.ndarray,
    labels: List,
    categories: List[str] = ["vel", "ego_euc", "ang", "avel"],
    n_pcs: int = 10,
    downsample: int = 1,
    method="fbpca",
):
    print("Calculating principal components ... ")

    # Initializing the PCA method
    # if method.startswith("torch"):
    #     import torch

    #     pca_feats = torch.zeros(features.shape[0], len(categories) * n_pcs)
    #     features = torch.tensor(features)
    # else:
    # Centering the features if not torch (pytorch does it itself)
    features = features - features.mean(axis=0)
    pca_feats = np.zeros(
        (features.shape[0], len(categories) * n_pcs), dtype=features.dtype
    )

    if method == "ipca":
        from sklearn.decomposition import IncrementalPCA

        pca = IncrementalPCA(n_components=n_pcs, batch_size=None)
    elif method.startswith("fbpca"):
        import fbpca

    num_cols = 0
    for i, cat in enumerate(tqdm(categories)):  # Iterate through each feature category
        cat += "_"
    
        cols_idx = [
            i
            for i, col in enumerate(labels)
            if (col.startswith(cat) or ("_" + cat in col))
        ]
        num_cols += len(cols_idx)
        print(f"Category: {cat}, Matching labels: {[label for label in combined_labels if cat in label]}")
        if not cols_idx:
            print(f"Skipping category {cat}: no matching columns found.")
            continue

        if method == "ipca" or method == "sklearn_pca":
            # import pdb; pdb.set_trace()
            pca_feats[:, i * n_pcs : (i + 1) * n_pcs] = pca.fit_transform(
                features[:, cols_idx]
            )

        # elif method.startswith("torch"):
        #     feat_cat = features[:, cols_idx]
        #     if method.endswith("_gpu"):
        #         feat_cat = feat_cat.cuda()

        #     if "pca" in method:
        #         (_, _, V) = torch.pca_lowrank(feat_cat)
        #     elif "svd" in method:
        #         feat_cat -= feat_cat.mean()
        #         (_, _, V) = torch.linalg.svd(feat_cat)

        #     if method.endswith("_gpu"):
        #         pca_feats[:, i * n_pcs : (i + 1) * n_pcs] = (
        #             torch.matmul(feat_cat, V[:, :n_pcs]).detach().cpu()
        #         )
        #         feat_cat.detach().cpu()
        #         V.detach().cpu()
        #     else:
        #         pca_feats[:, i * n_pcs : (i + 1) * n_pcs] = torch.matmul(
        #             feat_cat, V[:, :n_pcs]
        #         )

        elif method == "fbpca":
            print(f"Category: {cat}, Columns: {cols_idx}, Feature shape: {features[:, cols_idx].shape}")

            (_, _, V) = fbpca.pca(
                features[::downsample, cols_idx].astype(np.float64), k=n_pcs
            )
            pca_feats[:, i * n_pcs : (i + 1) * n_pcs] = np.matmul(
                features[:, cols_idx], V.astype(features.dtype).T
            )

    # if method.startswith("torch_pca"):
    #     pca_feats = pca_feats.numpy()

    # assert num_cols == features.shape[1]

    pc_labels = [
        "_".join([cat, "pc" + str(i)]) for cat in categories for i in range(n_pcs)
    ]

    return pca_feats, pc_labels

 
# Perform PCA on the combined features
pca_features, pca_labels = pca(
    features=combined_features,
    labels=combined_labels,
    categories=["vel", "avel", "ang", "ego_pose"],
    n_pcs=10,
    downsample=1,
    method="fbpca",
)

# Save or further analyze PCA results
print("PCA completed.")
print("PCA features shape:", pca_features.shape)
print("PCA labels:", pca_labels)

Calculating principal components ... 


  0%|          | 0/4 [00:00<?, ?it/s]

Category: vel_, Matching labels: ['rel_vel_norm_EarL_11', 'rel_vel_x_EarL_11', 'rel_vel_y_EarL_11', 'rel_vel_z_EarL_11', 'rel_vel_norm_EarR_11', 'rel_vel_x_EarR_11', 'rel_vel_y_EarR_11', 'rel_vel_z_EarR_11', 'rel_vel_norm_Snout_11', 'rel_vel_x_Snout_11', 'rel_vel_y_Snout_11', 'rel_vel_z_Snout_11', 'rel_vel_norm_SpineF_11', 'rel_vel_x_SpineF_11', 'rel_vel_y_SpineF_11', 'rel_vel_z_SpineF_11', 'rel_vel_norm_Tail(base)_11', 'rel_vel_x_Tail(base)_11', 'rel_vel_y_Tail(base)_11', 'rel_vel_z_Tail(base)_11', 'rel_vel_norm_Tail(mid)_11', 'rel_vel_x_Tail(mid)_11', 'rel_vel_y_Tail(mid)_11', 'rel_vel_z_Tail(mid)_11', 'rel_vel_norm_Tail(end)_11', 'rel_vel_x_Tail(end)_11', 'rel_vel_y_Tail(end)_11', 'rel_vel_z_Tail(end)_11', 'rel_vel_norm_ForepawL_11', 'rel_vel_x_ForepawL_11', 'rel_vel_y_ForepawL_11', 'rel_vel_z_ForepawL_11', 'rel_vel_norm_WristL_11', 'rel_vel_x_WristL_11', 'rel_vel_y_WristL_11', 'rel_vel_z_WristL_11', 'rel_vel_norm_ElbowL_11', 'rel_vel_x_ElbowL_11', 'rel_vel_y_ElbowL_11', 'rel_vel_z_

 25%|██▌       | 1/4 [00:28<01:24, 28.32s/it]

Category: avel_, Matching labels: ['avel_2_1_3_11', 'avel_2_0_3_11', 'avel_1_3_4_11', 'avel_0_3_4_11', 'avel_2_3_4_11', 'avel_3_4_5_11', 'avel_1_3_15_11', 'avel_0_3_15_11', 'avel_2_3_15_11', 'avel_4_3_15_11', 'avel_1_3_11_11', 'avel_0_3_11_11', 'avel_2_3_11_11', 'avel_4_3_11_11', 'avel_12_13_14_11', 'avel_13_14_15_11', 'avel_14_15_3_11', 'avel_8_9_10_11', 'avel_9_10_11_11', 'avel_10_11_3_11', 'avel_18_4_3_11', 'avel_18_4_5_11', 'avel_21_4_3_11', 'avel_21_4_5_11', 'avel_16_17_18_11', 'avel_17_18_4_11', 'avel_19_20_21_11', 'avel_20_21_4_11', 'avel_2_3_8_11', 'avel_2_3_9_11', 'avel_2_3_10_11', 'avel_2_3_12_11', 'avel_2_3_13_11', 'avel_2_3_14_11', 'avel_5_4_16_11', 'avel_5_4_17_11', 'avel_5_4_19_11', 'avel_5_4_20_11', 'avel_4_5_16_11', 'avel_4_5_17_11', 'avel_4_5_19_11', 'avel_4_5_20_11', 'avel_2_3_5_11', 'avel_2_4_16_11', 'avel_2_4_17_11', 'avel_2_4_18_11', 'avel_2_4_19_11', 'avel_2_4_20_11', 'avel_2_4_21_11', 'avel_2_0_16_11', 'avel_2_0_17_11', 'avel_2_0_18_11', 'avel_2_1_19_11', 'avel_2

 50%|█████     | 2/4 [00:50<00:49, 24.70s/it]

Category: ang_, Matching labels: ['ang_2_1_3', 'ang_2_0_3', 'ang_1_3_4', 'ang_0_3_4', 'ang_2_3_4', 'ang_3_4_5', 'ang_1_3_15', 'ang_0_3_15', 'ang_2_3_15', 'ang_4_3_15', 'ang_1_3_11', 'ang_0_3_11', 'ang_2_3_11', 'ang_4_3_11', 'ang_12_13_14', 'ang_13_14_15', 'ang_14_15_3', 'ang_8_9_10', 'ang_9_10_11', 'ang_10_11_3', 'ang_18_4_3', 'ang_18_4_5', 'ang_21_4_3', 'ang_21_4_5', 'ang_16_17_18', 'ang_17_18_4', 'ang_19_20_21', 'ang_20_21_4', 'ang_2_3_8', 'ang_2_3_9', 'ang_2_3_10', 'ang_2_3_12', 'ang_2_3_13', 'ang_2_3_14', 'ang_5_4_16', 'ang_5_4_17', 'ang_5_4_19', 'ang_5_4_20', 'ang_4_5_16', 'ang_4_5_17', 'ang_4_5_19', 'ang_4_5_20', 'ang_2_3_5', 'ang_2_4_16', 'ang_2_4_17', 'ang_2_4_18', 'ang_2_4_19', 'ang_2_4_20', 'ang_2_4_21', 'ang_2_0_16', 'ang_2_0_17', 'ang_2_0_18', 'ang_2_1_19', 'ang_2_1_20', 'ang_2_1_21']
Category: ang_, Columns: [738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 7

100%|██████████| 4/4 [00:55<00:00, 13.98s/it]

Category: ego_pose_, Matching labels: []
Skipping category ego_pose_: no matching columns found.
PCA completed.
PCA features shape: (3539000, 40)
PCA labels: ['vel_pc0', 'vel_pc1', 'vel_pc2', 'vel_pc3', 'vel_pc4', 'vel_pc5', 'vel_pc6', 'vel_pc7', 'vel_pc8', 'vel_pc9', 'avel_pc0', 'avel_pc1', 'avel_pc2', 'avel_pc3', 'avel_pc4', 'avel_pc5', 'avel_pc6', 'avel_pc7', 'avel_pc8', 'avel_pc9', 'ang_pc0', 'ang_pc1', 'ang_pc2', 'ang_pc3', 'ang_pc4', 'ang_pc5', 'ang_pc6', 'ang_pc7', 'ang_pc8', 'ang_pc9', 'ego_pose_pc0', 'ego_pose_pc1', 'ego_pose_pc2', 'ego_pose_pc3', 'ego_pose_pc4', 'ego_pose_pc5', 'ego_pose_pc6', 'ego_pose_pc7', 'ego_pose_pc8', 'ego_pose_pc9']


In [44]:
write.features_h5(pca_features, pca_labels, path=config["out_path"] + "_combined_pca.h5")

In [45]:
pc_feats, pc_labels = pca_features, pca_labels

wlet_feats, wlet_labels = features.wavelet(
    pc_feats, pc_labels, ids, f_s=30, freq=np.linspace(1, 25, 25), w0=5
)

write.features_h5(wlet_feats, wlet_labels, path=config["out_path"] + "_wavelets.h5")


Calculating wavelets ... 
Calculating wavelets for video 0.0


100%|██████████| 40/40 [00:01<00:00, 27.79it/s]


Calculating wavelets for video 1.0


100%|██████████| 40/40 [00:01<00:00, 28.09it/s]


Calculating wavelets for video 2.0


...

100%|██████████| 40/40 [00:00<00:00, 46.20it/s]


Calculating wavelets for video 113.0


100%|██████████| 40/40 [00:00<00:00, 88.56it/s]


Calculating wavelets for video 114.0


100%|██████████| 40/40 [00:00<00:00, 46.10it/s]


In [ ]:
# PCA on wavelet features
pc_wlet, pc_wlet_labels = features.pca(
    wlet_feats,
    wlet_labels,
    # categories=["wlet_ego_euc"],
    categories=["wlet_ang"],
    n_pcs=5,
    method="fbpca",
)

del wlet_feats, wlet_labels
pc_feats = np.hstack((pc_feats, pc_wlet))
pc_labels += pc_wlet_labels
del pc_wlet, pc_wlet_labels

write.features_h5(
    pc_feats, pc_labels, path="".join([config["out_path"], "_pca_on_wavelets.h5"])
)




data_obj = ds.DataStruct(
    pose=pose,
    id=ids,
    meta=meta,
    meta_by_frame=meta_by_frame,
    connectivity=connectivity,
)

data_obj.features = pc_feats
# When using high framerate data, downsampling may be necessary in order to 
# discover granular structure in embedding
data_obj = data_obj[:: config["downsample"], :]


embedder = Embed(
    embed_method=config["single_embed"]["method"],
    perplexity=config["single_embed"]["perplexity"],
    lr=config["single_embed"]["lr"],
)
data_obj.embed_vals = embedder.embed(data_obj.features, save_self=True)

# Watershed clustering
data_obj.ws = Watershed(
    sigma=config["single_embed"]["sigma"], max_clip=1, log_out=True, pad_factor=0.05
)
data_obj.data["Cluster"] = data_obj.ws.fit_predict(data=data_obj.embed_vals)
print("Writing Data Object to pickle")
data_obj.write_pickle(''.join([config['out_path'],'/']))



freq, combined_keys = analysis.cluster_freq_by_cat(
                        data_obj.data["Cluster"].values, cat=data_obj.id
                    )
freq_df = pd.DataFrame(freq.T, columns=combined_keys)
freq_df.to_csv(''.join([config['out_path'],'/cluster_occupancy.csv']))

vis.pose.sample_grid3D(
    pose-pose.mean(axis=-2, keepdims=True),
    connectivity=connectivity,
    
    labels=data_obj.data["Cluster"],
    n_samples=9,
    centered=True,
    N_FRAMES=100,
    fps=30,
    dpi=100,
    watershed=data_obj.ws,
    embed_vals=None,
    VID_NAME = "cluster",
    filepath=config["out_path"],
)

Calculating principal components ... 


100%|██████████| 1/1 [00:20<00:00, 20.49s/it]


Running fitsne via openTSNE
--------------------------------------------------------------------------------
TSNE(early_exaggeration=12, exaggeration=1.5, n_jobs=-1,
     negative_gradient_method='fft', neighbors='annoy', perplexity=50,
     verbose=True)
--------------------------------------------------------------------------------
===> Finding 150 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 66.30 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 10.18 seconds
===> Calculating PCA-based initialization...
   --> Time elapsed: 0.06 seconds
===> Running optimization with exaggeration=12.00, lr=29491.67 for 250 iterations...
Iteration   50, KL divergence 7.1468, 50 iterations in 3.3233 sec
Iteration  100, KL divergence 7.1304, 50 iterations in 3.3146 sec
Iteration  150, KL divergence 7.1191, 50 iterations in 3.3814 sec
Iteration  200, KL divergence 7.1161, 50 iterations in 3.2285 sec
Iteration  250, KL divergence 7.114

100%|██████████| 115/115 [00:00<00:00, 8417.30it/s]


Detected labels not the same shape as pose...
Assuming labels downsampled by 10


...

[1032630, 1376120, 477550, 3187270, 2910350, 1862610, 1087000, 854130, 508760]


 37%|███▋      | 31/83 [17:36<29:47, 34.37s/it]

[1952850, 2851530, 945180, 3026690, 317110, 2985250, 2890250, 2647960, 3402010]


 39%|███▊      | 32/83 [18:11<29:13, 34.38s/it]

[3527230, 2415590, 2454200, 3285450, 2300550, 2311780, 1713080, 2392080, 3536940]


In [43]:
# import yaml
# from pathlib import Path
# import subprocess
# import numpy as np
# from neuroposelib import read, write, features

# Path to precomputed features
base_out_path = config["out_path"]
#"/home/lq53/mir_repos/dappy_24_nov/byws_version/250109_opti_combinations/"
feature_files = {
    "ego_pose": base_out_path + "ego_pose.h5",
    "angle": base_out_path + "angels.h5",
    "velocity": base_out_path + "velocity.h5",
    "angular_velocity": base_out_path + "angular_velocity.h5",
    "head_angular": base_out_path + "head_angular.h5",
    "euler_angles": base_out_path + "euler_angles.h5",
}

# Feature combinations to test
# feature_combinations = [
#     ["ego_pose", "angle", "velocity"], #0
#     ["ego_pose", "angle", "velocity", "angular_velocity"], #combo1
#     ["ego_pose", "angle", "velocity", "head_angular"],
#     ["ego_pose", "angle", "velocity", "euler_angles"],
# ]
feature_combinations = [
    ["ego_pose", "angle"],
    ["velocity"],
    ["angular_velocity"],
]

# Iterate over combinations
for i, combination in enumerate(feature_combinations):
    # Set up paths
    out_path = f"{base_out_path}combo_{i}/"
    Path(out_path).mkdir(parents=True, exist_ok=True)

    # Load and combine selected features
    combined_features = []
    combined_labels = []

    for feature in combination:
        # features_data, labels = read.features_h5(feature_files[feature])
        # combined_features.append(features_data)
        # combined_labels += labels
        
        # features_data, labels = read.features_h5(feature_files[feature])
        # combined_features.append(features_data)

        # # Repeat labels for each row in the feature
        # labels_repeated = np.tile(labels, (features_data.shape[0] // len(labels), 1))
        # combined_labels.append(labels_repeated)

        features_data, labels = read.features_h5(feature_files[feature])
        combined_features.append(features_data)

    combined_features = np.hstack(combined_features)
    # combined_labels = np.vstack(combined_labels)


    # Perform PCA
    pc_feats, pc_labels = pca(
        combined_features,
        combined_labels,
        categories=combination,
        n_pcs=10,
        method="fbpca",
    )
    write.features_h5(pc_feats, pc_labels, path=out_path + "pca_features.h5")

    # Wavelet transformation
    wlet_feats, wlet_labels = features.wavelet(
        pc_feats, pc_labels, ids=None, f_s=30, freq=np.linspace(1, 25, 25), w0=5
    )
    write.features_h5(wlet_feats, wlet_labels, path=out_path + "wavelets.h5")

    # PCA on wavelet features
    pc_wlet, pc_wlet_labels = features.pca(
        wlet_feats, wlet_labels, categories=["wlet_ang"], n_pcs=5, method="fbpca"
    )
    del wlet_feats, wlet_labels
    pc_feats = np.hstack((pc_feats, pc_wlet))
    pc_labels += pc_wlet_labels
    del pc_wlet, pc_wlet_labels

    
    write.features_h5(pc_feats, pc_labels, path=out_path + "pca_on_wavelets.h5")

    # Embedding and clustering
    data_obj = ds.DataStruct(
    pose=pose,
    id=ids,
    meta=meta,
    meta_by_frame=meta_by_frame,
    connectivity=connectivity,
    )


    data_obj.features = pc_feats
    data_obj = data_obj[:: config["downsample"], :]

    embedder = Embed(
        embed_method=config["single_embed"]["method"],
        perplexity=config["single_embed"]["perplexity"],
        lr=config["single_embed"]["lr"],
    )
    data_obj.embed_vals = embedder.embed(data_obj.features, save_self=True)

    data_obj.ws = Watershed(
        sigma=config["single_embed"]["sigma"], max_clip=1, log_out=True, pad_factor=0.05
    )
    data_obj.data["Cluster"] = data_obj.ws.fit_predict(data=data_obj.embed_vals)
    data_obj.write_pickle(out_path + "data_object.pkl")

    # Save cluster frequencies
    freq, combined_keys = analysis.cluster_freq_by_cat(
        data_obj.data["Cluster"].values, cat=data_obj.id
    )
    freq_df = pd.DataFrame(freq.T, columns=combined_keys)
    freq_df.to_csv(out_path + "cluster_occupancy.csv")

    # Visualization
    vis.pose.sample_grid3D(
    pose-pose.mean(axis=-2, keepdims=True),
    connectivity=connectivity,
    labels=data_obj.data["Cluster"],
    n_samples=9,
    centered=True,
    N_FRAMES=100,
    fps=30,
    dpi=100,
    watershed=data_obj.ws,
    embed_vals=None,
    VID_NAME = "cluster",
    filepath=config["out_path"],
    )

    print(f"Combination {i}: {combination} completed and saved to {out_path}.")

print("All combinations processed.")

Features loaded at path /home/lq53/mir_repos/dappy_24_nov/byws_version/250109_opti_combb/50_p/ego_pose.h5
Features loaded at path /home/lq53/mir_repos/dappy_24_nov/byws_version/250109_opti_combb/50_p/angels.h5
Calculating principal components ... 


100%|██████████| 2/2 [00:00<00:00, 31068.92it/s]


Category: ego_pose_, Matching labels: []
Skipping category ego_pose_: no matching columns found.
Category: angle_, Matching labels: []
Skipping category angle_: no matching columns found.
Calculating wavelets ... 
Calculating wavelets for video None


  0%|          | 0/20 [00:00<?, ?it/s]


ValueError: volume and kernel should have the same dimensionality

In [27]:
print(f"Combination {i}: {combination}")
print(f"Combined features shape: {combined_features.shape}")


Combination 0: ['ego_pose', 'angle']
Combined features shape: (3539000, 121)


In [20]:
for feature in combination:
    features_data, labels = read.features_h5(feature_files[feature])
    print(f"Feature: {feature}, Labels shape: {labels}")


Features loaded at path /home/lq53/mir_repos/dappy_24_nov/byws_version/250109_opti_combb/50_p/ego_pose.h5
Feature: ego_pose, Labels shape: ['ego_euc_EarL_x', 'ego_euc_EarL_y', 'ego_euc_EarL_z', 'ego_euc_EarR_x', 'ego_euc_EarR_y', 'ego_euc_EarR_z', 'ego_euc_Snout_x', 'ego_euc_Snout_y', 'ego_euc_Snout_z', 'ego_euc_SpineF_x', 'ego_euc_SpineF_y', 'ego_euc_SpineF_z', 'ego_euc_SpineM_x', 'ego_euc_SpineM_y', 'ego_euc_SpineM_z', 'ego_euc_Tail(base)_x', 'ego_euc_Tail(base)_y', 'ego_euc_Tail(base)_z', 'ego_euc_Tail(mid)_x', 'ego_euc_Tail(mid)_y', 'ego_euc_Tail(mid)_z', 'ego_euc_Tail(end)_x', 'ego_euc_Tail(end)_y', 'ego_euc_Tail(end)_z', 'ego_euc_ForepawL_x', 'ego_euc_ForepawL_y', 'ego_euc_ForepawL_z', 'ego_euc_WristL_x', 'ego_euc_WristL_y', 'ego_euc_WristL_z', 'ego_euc_ElbowL_x', 'ego_euc_ElbowL_y', 'ego_euc_ElbowL_z', 'ego_euc_ShoulderL_x', 'ego_euc_ShoulderL_y', 'ego_euc_ShoulderL_z', 'ego_euc_ForepawR_x', 'ego_euc_ForepawR_y', 'ego_euc_ForepawR_z', 'ego_euc_WristR_x', 'ego_euc_WristR_y', 'ego

In [16]:
print(f"Combined features shape: {combined_features.shape}")
print(f"Combined labels length: {len(combined_labels)}")
assert combined_features.shape[0] == len(combined_labels), "Mismatch in feature and label dimensions"


Combined features shape: (3539000, 529)
Combined labels length: 540


AssertionError: Mismatch in feature and label dimensions

In [22]:
for feature in feature_files.keys():
    _, labels = read.features_h5(feature_files[feature])
    print(f"Feature: {feature}, Number of Labels: {len(labels)}")
    print(f"Labels: {labels[:10]}...")  # Print a preview of the first 10 labels
    print("-" * 50)


Features loaded at path /home/lq53/mir_repos/dappy_24_nov/byws_version/250109_opti_combb/50_p/ego_pose.h5
Feature: ego_pose, Number of Labels: 66
Labels: ['ego_euc_EarL_x', 'ego_euc_EarL_y', 'ego_euc_EarL_z', 'ego_euc_EarR_x', 'ego_euc_EarR_y', 'ego_euc_EarR_z', 'ego_euc_Snout_x', 'ego_euc_Snout_y', 'ego_euc_Snout_z', 'ego_euc_SpineF_x']...
--------------------------------------------------
Features loaded at path /home/lq53/mir_repos/dappy_24_nov/byws_version/250109_opti_combb/50_p/angels.h5
Feature: angle, Number of Labels: 66
Labels: ['ego_euc_EarL_x', 'ego_euc_EarL_y', 'ego_euc_EarL_z', 'ego_euc_EarR_x', 'ego_euc_EarR_y', 'ego_euc_EarR_z', 'ego_euc_Snout_x', 'ego_euc_Snout_y', 'ego_euc_Snout_z', 'ego_euc_SpineF_x']...
--------------------------------------------------
Features loaded at path /home/lq53/mir_repos/dappy_24_nov/byws_version/250109_opti_combb/50_p/velocity.h5
Feature: velocity, Number of Labels: 408
Labels: ['abs_vel_norm_EarL_11', 'abs_vel_x_EarL_11', 'abs_vel_y_EarL_